<a href="https://colab.research.google.com/github/fleanend/TorchGlocalK/blob/main/Glocal_K.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from time import time
from scipy.sparse import csc_matrix
import numpy as np
import h5py
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torch.nn.parameter import Parameter

torch.manual_seed(1284)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data Loader Function

In [2]:
import pandas as pd
p='/home/recordk/reco/BX-Book-Ratings.csv'
a=pd.read_csv(p)

In [3]:
len(a[a['Book-Rating']!=0]) *0.3

10656.6

In [4]:
print(len(a),len(a[a['Book-Rating']==0]),len(a[a['Book-Rating']!=0]))

100726 65204 35522


In [5]:
test=a[a['Book-Rating']!=0]
test_x=test.drop('Book-Rating',axis=1)
test_y=test['Book-Rating']

In [6]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le2=LabelEncoder()
test_x['ISBN']=le.fit_transform(test_x['ISBN'])
test_x['User-ID']=le2.fit_transform(test_x['User-ID'])


In [7]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test=train_test_split(test_x,test_y,test_size=0.3)

In [8]:
tr=pd.concat([x_train,y_train],axis=1).values

In [9]:
# tr=pd.concat([x_train,y_train],axis=1).values
# tr=pd.concat([tr,a[a['Book-Rating']==0]]).values

In [10]:
tr

array([[ 4745, 16706,     5],
       [ 4242, 18315,     7],
       [ 2542,  6440,    10],
       ...,
       [ 1002, 18946,     8],
       [ 1956, 17899,     5],
       [  635, 13057,     6]])

In [11]:
te=pd.concat([x_test,y_test],axis=1).values

In [12]:
te

array([[   98,  9542,     8],
       [ 1940,  3865,     9],
       [ 3996,  4924,     8],
       ...,
       [  858, 15141,     9],
       [ 3935,  2082,     9],
       [ 3371,    44,    10]])

In [13]:
a.values[:,0]

array([276762, 276762, 276762, ..., 276579, 276579, 276579], dtype=object)

In [14]:
def load_data_100k(path='./data/', delimiter='\t'):

    # train = np.loadtxt(path+'movielens_100k_u1.base', skiprows=0, delimiter=delimiter).astype('int32')
    # test = np.loadtxt(path+'movielens_100k_u1.test', skiprows=0, delimiter=delimiter).astype('int32')
    train =tr
    test=te
    total = np.concatenate((train, test), axis=0)
    print(train)
    print(test)
    print(total)
    n_u = np.unique(total[:,0]).size  # num of users
    n_m = np.unique(total[:,1]).size  # num of movies
    n_train = train.shape[0]  # num of training ratings
    n_test = test.shape[0]  # num of test ratings

    train_r = np.zeros((n_m, n_u), dtype='float32')
    test_r = np.zeros((n_m, n_u), dtype='float32')

    for i in range(n_train):
        train_r[train[i,1]-1, train[i,0]-1] = train[i,2]

    for i in range(n_test):
        test_r[test[i,1]-1, test[i,0]-1] = test[i,2]

    train_m = np.greater(train_r, 1e-12).astype('float32')  # masks indicating non-zero entries
    test_m = np.greater(test_r, 1e-12).astype('float32')

    print('data matrix loaded')
    print('num of users: {}'.format(n_u))
    print('num of movies: {}'.format(n_m))
    print('num of training ratings: {}'.format(n_train))
    print('num of test ratings: {}'.format(n_test))

    return n_m, n_u, train_r, train_m, test_r, test_m

# Load Data ex

In [15]:
# Insert the path of a data directory by yourself (e.g., '/content/.../data')
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
data_path = ''
# .-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._

In [16]:
# Data Load
try:
    path = data_path + '/content/Glocal_K/data/MovieLens_100K/'
    n_m, n_u, train_r, train_m, test_r, test_m = load_data_100k(delimiter='\t')
except Exception:
    print('Error: Unable to load data')

[[ 4745 16706     5]
 [ 4242 18315     7]
 [ 2542  6440    10]
 ...
 [ 1002 18946     8]
 [ 1956 17899     5]
 [  635 13057     6]]
[[   98  9542     8]
 [ 1940  3865     9]
 [ 3996  4924     8]
 ...
 [  858 15141     9]
 [ 3935  2082     9]
 [ 3371    44    10]]
[[ 4745 16706     5]
 [ 4242 18315     7]
 [ 2542  6440    10]
 ...
 [  858 15141     9]
 [ 3935  2082     9]
 [ 3371    44    10]]


data matrix loaded
num of users: 4777
num of movies: 22222
num of training ratings: 24865
num of test ratings: 10657


# Load Data

In [17]:
n_m, n_u, train_r, train_m, test_r, test_m = load_data_100k(delimiter='\t')

[[ 4745 16706     5]
 [ 4242 18315     7]
 [ 2542  6440    10]
 ...
 [ 1002 18946     8]
 [ 1956 17899     5]
 [  635 13057     6]]
[[   98  9542     8]
 [ 1940  3865     9]
 [ 3996  4924     8]
 ...
 [  858 15141     9]
 [ 3935  2082     9]
 [ 3371    44    10]]
[[ 4745 16706     5]
 [ 4242 18315     7]
 [ 2542  6440    10]
 ...
 [  858 15141     9]
 [ 3935  2082     9]
 [ 3371    44    10]]
data matrix loaded
num of users: 4777
num of movies: 22222
num of training ratings: 24865
num of test ratings: 10657


In [18]:
# Common hyperparameter settings
n_hid = 500 # size of hidden layers
n_dim = 5 # inner AE embedding size
n_layers = 2 # number of hidden layers
gk_size = 3 # width=height of kernel for convolution
    
# Hyperparameters to tune for specific case
max_epoch_p = 500 # max number of epochs for pretraining
max_epoch_f = 1000 # max number of epochs for finetuning
patience_p = 5 # number of consecutive rounds of early stopping condition before actual stop for pretraining
patience_f = 10 # and finetuning
tol_p = 1e-4 # minimum threshold for the difference between consecutive values of train rmse, used for early stopping, for pretraining
tol_f = 1e-5 # and finetuning
lambda_2 = 20. # regularisation of number or parameters
lambda_s = 0.006 # regularisation of sparsity of the final matrix
dot_scale = 1 # dot product weight for global kernel

# Network Functions

In [19]:
def local_kernel(u, v):
    dist = torch.norm(u - v, p=2, dim=2)
    hat = torch.clamp(1. - dist**2, min=0.)
    return hat

class KernelLayer(nn.Module):
    def __init__(self, n_in, n_hid, n_dim, lambda_s, lambda_2, activation=nn.Sigmoid()):
      super().__init__()
      self.W = nn.Parameter(torch.randn(n_in, n_hid))
      self.u = nn.Parameter(torch.randn(n_in, 1, n_dim))
      self.v = nn.Parameter(torch.randn(1, n_hid, n_dim))
      self.b = nn.Parameter(torch.randn(n_hid))

      self.lambda_s = lambda_s
      self.lambda_2 = lambda_2

      nn.init.xavier_uniform_(self.W, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.xavier_uniform_(self.u, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.xavier_uniform_(self.v, gain=torch.nn.init.calculate_gain("relu"))
      nn.init.zeros_(self.b)
      self.activation = activation

    def forward(self, x):
      w_hat = local_kernel(self.u, self.v)
    
      sparse_reg = torch.nn.functional.mse_loss(w_hat, torch.zeros_like(w_hat))
      sparse_reg_term = self.lambda_s * sparse_reg
      
      l2_reg = torch.nn.functional.mse_loss(self.W, torch.zeros_like(self.W))
      l2_reg_term = self.lambda_2 * l2_reg

      W_eff = self.W * w_hat  # Local kernelised weight matrix
      y = torch.matmul(x, W_eff) + self.b
      y = self.activation(y)

      return y, sparse_reg_term + l2_reg_term

class KernelNet(nn.Module):
    def __init__(self, n_u, n_hid, n_dim, n_layers, lambda_s, lambda_2):
      super().__init__()
      layers = []
      for i in range(n_layers):
        if i == 0:
          layers.append(KernelLayer(n_u, n_hid, n_dim, lambda_s, lambda_2))
        else:
          layers.append(KernelLayer(n_hid, n_hid, n_dim, lambda_s, lambda_2))
      layers.append(KernelLayer(n_hid, n_u, n_dim, lambda_s, lambda_2, activation=nn.Identity()))
      self.layers = nn.ModuleList(layers)
      self.dropout = nn.Dropout(0.33)

    def forward(self, x):
      total_reg = None
      for i, layer in enumerate(self.layers):
        x, reg = layer(x)
        if i < len(self.layers)-1:
          x = self.dropout(x)
        if total_reg is None:
          total_reg = reg
        else:
          total_reg += reg
      return x, total_reg

In [20]:
class CompleteNet(nn.Module):
    def __init__(self, kernel_net, n_u, n_m, n_hid, n_dim, n_layers, lambda_s, lambda_2, gk_size, dot_scale):
      super().__init__()
      self.gk_size = gk_size
      self.dot_scale = dot_scale
      self.local_kernel_net = kernel_net
      # self.local_kernel_net = nn.ModuleList([kernel_net])

      self.conv_kernel = torch.nn.Parameter(torch.randn(n_m, gk_size**2) * 0.1)
      nn.init.xavier_uniform_(self.conv_kernel, gain=torch.nn.init.calculate_gain("relu"))
      

    def forward(self, x, x_local):
      gk = self.global_kernel(x_local, self.gk_size, self.dot_scale)
      x = self.global_conv(x, gk)
      x, global_reg_loss = self.local_kernel_net(x)
      return x, global_reg_loss

    def global_kernel(self, input, gk_size, dot_scale):
      avg_pooling = torch.mean(input, dim=1)  # Item (axis=1) based average pooling
      avg_pooling = avg_pooling.view(1, -1)
      # print('#'*10,avg_pooling.shape, self.conv_kernel.shape)
      gk = torch.matmul(avg_pooling, self.conv_kernel) * dot_scale  # Scaled dot product
      gk = gk.view(1, 1, gk_size, gk_size)

      return gk

    def global_conv(self, input, W):
      input = input.unsqueeze(0).unsqueeze(0)
      conv2d = nn.LeakyReLU()(F.conv2d(input, W, stride=1, padding=1))
      return conv2d.squeeze(0).squeeze(0)

class Loss(nn.Module):
    def forward(self, pred_p, reg_loss, train_m, train_r):
      # L2 loss
      diff = train_m * (train_r - pred_p)
      sqE = torch.nn.functional.mse_loss(diff, torch.zeros_like(diff))
      loss_p = sqE + reg_loss
      return loss_p

# Network Instantiation

## Pre-training

In [21]:
[f'cuda:{i}' for i in [0,1,2,3]]

['cuda:0', 'cuda:1', 'cuda:2', 'cuda:3']

In [22]:
device =torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = KernelNet(n_u, n_hid, n_dim, n_layers, lambda_s, lambda_2).double().to(device)
# Check if multiple GPUs are available
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = nn.DataParallel(model,device_ids=[0,1,2,3])

# Move the model to GPU
# model = model.to(device)
model = model.to(device)

## Fine-tuning

In [23]:
complete_model = CompleteNet(model, n_u, n_m, n_hid, n_dim, n_layers, lambda_s, lambda_2, gk_size, dot_scale).double().to(device)

# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     complete_model = nn.DataParallel(complete_model,device_ids=[0,1,2,3])

# Move the model to GPU
complete_model = complete_model.to(device)

In [24]:
device

device(type='cuda', index=1)

# Evaluation code

In [25]:
def dcg_k(score_label, k):
    dcg, i = 0., 0
    for s in score_label:
        if i < k:
            dcg += (2**s[1]-1) / np.log2(2+i)
            i += 1
    return dcg

In [26]:
def ndcg_k(y_hat, y, k):
    score_label = np.stack([y_hat, y], axis=1).tolist()
    score_label = sorted(score_label, key=lambda d:d[0], reverse=True)
    score_label_ = sorted(score_label, key=lambda d:d[1], reverse=True)
    norm, i = 0., 0
    for s in score_label_:
        if i < k:
            norm += (2**s[1]-1) / np.log2(2+i)
            i += 1
    dcg = dcg_k(score_label, k)
    return dcg / norm

In [27]:
def call_ndcg(y_hat, y):
    ndcg_sum, num = 0, 0
    y_hat, y = y_hat.T, y.T
    n_users = y.shape[0]

    for i in range(n_users):
        y_hat_i = y_hat[i][np.where(y[i])]
        y_i = y[i][np.where(y[i])]

        if y_i.shape[0] < 2:
            continue

        ndcg_sum += ndcg_k(y_hat_i, y_i, y_i.shape[0])  # user-wise calculation
        num += 1

    return ndcg_sum / num

# Training and Test Loop

In [28]:
best_rmse_ep, best_mae_ep, best_ndcg_ep = 0, 0, 0
best_rmse, best_mae, best_ndcg = float("inf"), float("inf"), 0

time_cumulative = 0
tic = time()

# Pre-Training
optimizer = torch.optim.AdamW(complete_model.local_kernel_net.parameters(), lr=0.001)
# optimizer = torch.optim.AdamW(complete_model.module.local_kernel_net.parameters(), lr=0.001)

def closure():
  optimizer.zero_grad()
  x = torch.Tensor(train_r).double().to(device)
  m = torch.Tensor(train_m).double().to(device)
  
  complete_model.local_kernel_net.train()
  # complete_model.module.local_kernel_net.train()
  
  pred, reg = complete_model.local_kernel_net(x)
  # pred, reg = complete_model.module.local_kernel_net(x)
  loss = Loss().to(device)(pred, reg, m, x)
  loss.backward()
  return loss

last_rmse = np.inf
counter = 0

for i in range(max_epoch_p):
  optimizer.step(closure)
  
  complete_model.local_kernel_net.eval()
  # complete_model.module.local_kernel_net.eval()
  t = time() - tic
  time_cumulative += t

  pre, _ = model(torch.Tensor(train_r).double().to(device))
  
  pre = pre.float().cpu().detach().numpy()
  
  error = (test_m * (np.clip(pre, 1., 5.) - test_r) ** 2).sum() / test_m.sum()  # test error
  test_rmse = np.sqrt(error)

  error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()  # train error
  train_rmse = np.sqrt(error_train)

  if last_rmse-train_rmse < tol_p:
    counter += 1
  else:
    counter = 0

  last_rmse = train_rmse

  if patience_p == counter:
    print('.-^-._' * 12)
    print('PRE-TRAINING')
    print('Epoch:', i+1, 'test rmse:', test_rmse, 'train rmse:', train_rmse)
    print('Time:', t, 'seconds')
    print('Time cumulative:', time_cumulative, 'seconds')
    print('.-^-._' * 12)
    break


  if i % 50 != 0:
    continue
  print('.-^-._' * 12)
  print('PRE-TRAINING')
  print('Epoch:', i, 'test rmse:', test_rmse, 'train rmse:', train_rmse)
  print('Time:', t, 'seconds')
  print('Time cumulative:', time_cumulative, 'seconds')
  print('.-^-._' * 12)

.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 0 test rmse: 6.986413 train rmse: 6.9819584
Time: 1.1739418506622314 seconds
Time cumulative: 1.1739418506622314 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 50 test rmse: 4.2616973 train rmse: 3.939104
Time: 137.5244755744934 seconds
Time cumulative: 3537.6545436382294 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
PRE-TRAINING
Epoch: 75 test rmse: 4.1802683 train rmse: 3.8537848
Time: 202.9475908279419 seconds
Time cumulative: 7655.298001527786 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._


In [29]:
# Fine-Tuning

train_r_local = np.clip(pre, 1., 5.)

optimizer = torch.optim.AdamW(complete_model.parameters(), lr=0.001)
# complete_model=nn.DataParallel(complete_model)
def closure():
  optimizer.zero_grad()
  x = torch.Tensor(train_r).double().to(device)
  x_local = torch.Tensor(train_r_local).double().to(device)
  m = torch.Tensor(train_m).double().to(device)
  complete_model.train()
  pred, reg = complete_model(x, x_local)
  loss = Loss().to(device)(pred, reg, m, x)
  loss.backward()
  return loss

last_rmse = np.inf
counter = 0

for i in range(max_epoch_f):
  optimizer.step(closure)
  complete_model.eval()
  t = time() - tic
  time_cumulative += t

  pre, _ = complete_model(torch.Tensor(train_r).double().to(device), torch.Tensor(train_r_local).double().to(device))
  
  pre = pre.float().cpu().detach().numpy()

  error = (test_m * (np.clip(pre, 1., 5.) - test_r) ** 2).sum() / test_m.sum()  # test error
  test_rmse = np.sqrt(error)

  error_train = (train_m * (np.clip(pre, 1., 5.) - train_r) ** 2).sum() / train_m.sum()  # train error
  train_rmse = np.sqrt(error_train)

  test_mae = (test_m * np.abs(np.clip(pre, 1., 5.) - test_r)).sum() / test_m.sum()
  train_mae = (train_m * np.abs(np.clip(pre, 1., 5.) - train_r)).sum() / train_m.sum()

  test_ndcg = call_ndcg(np.clip(pre, 1., 5.), test_r)
  train_ndcg = call_ndcg(np.clip(pre, 1., 5.), train_r)

  if test_rmse < best_rmse:
      best_rmse = test_rmse
      best_rmse_ep = i+1

  if test_mae < best_mae:
      best_mae = test_mae
      best_mae_ep = i+1

  if best_ndcg < test_ndcg:
      best_ndcg = test_ndcg
      best_ndcg_ep = i+1

  if last_rmse-train_rmse < tol_f:
    counter += 1
  else:
    counter = 0

  last_rmse = train_rmse

  if patience_f == counter:
    print('.-^-._' * 12)
    print('FINE-TUNING')
    print('Epoch:', i+1, 'test rmse:', test_rmse, 'test mae:', test_mae, 'test ndcg:', test_ndcg)
    print('Epoch:', i+1, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
    print('Time:', t, 'seconds')
    print('Time cumulative:', time_cumulative, 'seconds')
    print('.-^-._' * 12)
    break


  if i % 50 != 0:
    continue

  print('.-^-._' * 12)
  print('FINE-TUNING')
  print('Epoch:', i, 'test rmse:', test_rmse, 'test mae:', test_mae, 'test ndcg:', test_ndcg)
  print('Epoch:', i, 'train rmse:', train_rmse, 'train mae:', train_mae, 'train ndcg:', train_ndcg)
  print('Time:', t, 'seconds')
  print('Time cumulative:', time_cumulative, 'seconds')
  print('.-^-._' * 12)

########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) torch.Size([22222, 9])
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
FINE-TUNING
Epoch: 0 test rmse: 4.018015 test mae: 3.564016 test ndcg: 0.8734342013268793
Epoch: 0 train rmse: 3.6753216 train mae: 3.2868927 train ndcg: 0.8545042217177146
Time: 208.5083131790161 seconds
Time cumulative: 7863.806314706802 seconds
.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._.-^-._
########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) torch.Size([22222, 9])
########## torch.Size([1, 22222]) to

In [37]:
# Final result
print('Epoch:', best_rmse_ep, ' best rmse:', best_rmse)
print('Epoch:', best_mae_ep, ' best mae:', best_mae)
print('Epoch:', best_ndcg_ep, ' best ndcg:', best_ndcg)

Epoch: 7  best rmse: 3.988716
Epoch: 7  best mae: 3.543069
Epoch: 1  best ndcg: 0.8734342013268793


In [38]:
le.transform(['013022393X'])

array([34])

In [39]:
book=pd.read_csv('/home/recordk/reco/BX-Books.csv',encoding='CP949')

In [40]:
book

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,9643112136,Dalan-i bihisht (Dastan-i Irani),Nazi Safavi,1378,Intisharat-i Quqnus,http://images.amazon.com/images/P/9643112136.0...,http://images.amazon.com/images/P/9643112136.0...,http://images.amazon.com/images/P/9643112136.0...
1,1551103982,The Cycling Adventures of Coconut Head: A Nort...,Ted Schredd,1900,Graphic Arts Center Pub Co,http://images.amazon.com/images/P/1551103982.0...,http://images.amazon.com/images/P/1551103982.0...,http://images.amazon.com/images/P/1551103982.0...
2,013022393X,All the Best Salads and Salad Dressings,J. Warner,1911,Prentice Hall Direct,http://images.amazon.com/images/P/013022393X.0...,http://images.amazon.com/images/P/013022393X.0...,http://images.amazon.com/images/P/013022393X.0...
3,030758013X,Assorted Small Tray Puzzles,Golden,1920,Golden Books,http://images.amazon.com/images/P/030758013X.0...,http://images.amazon.com/images/P/030758013X.0...,http://images.amazon.com/images/P/030758013X.0...
4,082493069X,Grill and Barbecue Cooking,Ideals Publications Inc,1920,Ideals Publications,http://images.amazon.com/images/P/082493069X.0...,http://images.amazon.com/images/P/082493069X.0...,http://images.amazon.com/images/P/082493069X.0...
...,...,...,...,...,...,...,...,...
44360,B00007MF56,"More, Now, Again: A Memoir of Addiction",Elizabeth Wurtzel,2001,Simon &amp Schuster,http://images.amazon.com/images/P/B00007MF56.0...,http://images.amazon.com/images/P/B00007MF56.0...,http://images.amazon.com/images/P/B00007MF56.0...
44361,B00007CWGV,Madonna,Andrew Morton,2001,St. Martin's Press,http://images.amazon.com/images/P/B00007CWGV.0...,http://images.amazon.com/images/P/B00007CWGV.0...,http://images.amazon.com/images/P/B00007CWGV.0...
44362,B000078UH8,The Hearing,John T. Lescroart,2001,E P Dutton,http://images.amazon.com/images/P/B000078UH8.0...,http://images.amazon.com/images/P/B000078UH8.0...,http://images.amazon.com/images/P/B000078UH8.0...
44363,B00006JO6O,The Last Precinct,Patricia Daniels Cornwell,2000,Putnam Pub Group,http://images.amazon.com/images/P/B00006JO6O.0...,http://images.amazon.com/images/P/B00006JO6O.0...,http://images.amazon.com/images/P/B00006JO6O.0...


In [41]:
user=276579
a[(a['User-ID']==user)&(a['Book-Rating']>0)].index

Index([100722, 100724], dtype='int64')

In [42]:
a.drop(a[(a['User-ID']==user)&(a['Book-Rating']>0)].index)

,User-ID,ISBN,Book-Rating
0,276762,3404122879,0
1,276762,3404182928,0
2,276762,3426690179,0
3,276762,3442424216,0
4,276762,3442425573,0
...,...,...,...
100719,267609,3785715382,0
100720,267609,3785715390,0
100721,267609,3785721080,0
100723,276579,087579159X,0


In [ ]:
le.transform([user])
le2.transform()
complete_model()

In [44]:
pd.DataFrame(train_r)

,0,1,2,3,4,5,6,7,8,9,...,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22217,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22219,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# 추천하기
def recommender(user, n_items=10):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    # rated_index = a[a['Book-Rating']>0].index    # 이미 평가한 책 확인
    rated_index = a[(a['User-ID']==user)&(a['Book-Rating']>0)].index
    # items = a.loc[user].drop(rated_index)
    books = a.drop(rated_index)
    for book in books.index:
        # predictions.append(complete_model.get_one_prediction(le.transform([user]), le2.transform([book])))                   # 예상평점 계산
        predictions.append(complete_model(le.transform([user]), le2.transform([book])))                   # 예상평점 계산
    recommendations = pd.Series(data=predictions, index=books.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]    # 예상평점이 가장 높은 영화 선택
    recommended_items = book.loc[recommendations.index]['title']
    return recommended_items